In [60]:
import os
import sys
import codecs
import nltk
from nltk.tree import Tree
from scipy.stats import pearsonr

In [57]:
path_to_parse_trees = 'data/train-test.hyp1-hyp2-ref.parse'
path_to_labels = 'data/train.gold'

In [40]:
lines = [line.strip() for line in codecs.open(path_to_parse_trees, 'rb', encoding='utf8')]

In [58]:
labels = [int(line.strip()) for line in open(path_to_labels, 'r')]

In [41]:
curr_ind = 0
parses = []
for ind, line in enumerate(lines):
    if line == '':
        parses.append(lines[curr_ind:ind])
        curr_ind = ind + 1

In [42]:
parses = [''.join(parse) for parse in parses]

In [43]:
grouped_parses = []
for i in range(0, len(parses), 3):
    grouped_parses.append(parses[i:i+3])

In [44]:
def tree_kernel(parse1, parse2):
    parse1 = Tree.fromstring(parse1)
    parse2 = Tree.fromstring(parse2)
    
    parse1_subtrees = list(parse1.subtrees())
    parse2_subtrees = list(parse2.subtrees())
    
    score = 0
    for subtree_1 in parse1_subtrees:
        for subtree_2 in parse2_subtrees:
            if subtree_1 == subtree_2:
                score += len(subtree_1.leaves())
    score = score / 2.0 # Removing a = b / b = a occurrences
    score = float(score) / max(len(parse1_subtrees), len(parse2_subtrees))
    
    return score

In [47]:
scores = []
for ind, parses in enumerate(grouped_parses):
    if ind % 1000 == 0:
        print 'Finished %d sentence pairs of out %d ' % (ind, len(grouped_parses))
    scores.append([
            tree_kernel(parses[0], parses[2]),
            tree_kernel(parses[1], parses[2])
        ])

Finished 0 sentence pairs of out 50339 
Finished 1000 sentence pairs of out 50339 
Finished 2000 sentence pairs of out 50339 
Finished 3000 sentence pairs of out 50339 
Finished 4000 sentence pairs of out 50339 
Finished 5000 sentence pairs of out 50339 
Finished 6000 sentence pairs of out 50339 
Finished 7000 sentence pairs of out 50339 
Finished 8000 sentence pairs of out 50339 
Finished 9000 sentence pairs of out 50339 
Finished 10000 sentence pairs of out 50339 
Finished 11000 sentence pairs of out 50339 
Finished 12000 sentence pairs of out 50339 
Finished 13000 sentence pairs of out 50339 
Finished 14000 sentence pairs of out 50339 
Finished 15000 sentence pairs of out 50339 
Finished 16000 sentence pairs of out 50339 
Finished 17000 sentence pairs of out 50339 
Finished 18000 sentence pairs of out 50339 
Finished 19000 sentence pairs of out 50339 
Finished 20000 sentence pairs of out 50339 
Finished 21000 sentence pairs of out 50339 
Finished 22000 sentence pairs of out 50339 
F

In [66]:
f = open('data/features.parse', 'w')
for score in scores:
    f.write(' '.join([str(x) for x in score]) + '\n')
f.close()

In [69]:
pearsonr([score[1] - score[0] for score in scores][:len(labels)], labels)

(0.17118232855135804, 1.6368223785981011e-171)